In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp
import inspect

fpga = overlay()
rp.rp_Init()


DEF FUNCS

In [ ]:
def generate_single_pulse_wave( dcyc=1,Ampl=1):
    BUFFER_SIZE = 16384 
    t = np.linspace(0, 1, BUFFER_SIZE, endpoint=False)    
    period = 1 
    pulse_width = dcyc * period  
    waveform = np.where(t < pulse_width, Ampl, 0.0)  
    return  waveform

def gen_sine(Ampl=1):
    BUFFER_SIZE = 16384 
    t=np.linspace(0,np.pi,BUFFER_SIZE)
    ss=Ampl*np.sin(t)
    return ss

DEFINE PARAMS

In [ ]:
channel = rp.RP_CH_1        
waveform = rp.RP_WAVEFORM_SQUARE

freq=1e6
d=2
dec=2**d
#dec = rp.RP_DEC_8
duty=0.5
Amp=1

ncyc = 1
nor = 1
period =100
gen_trig_sour = rp.RP_GEN_TRIG_SRC_INTERNAL

trig_lvl = 0.1
trig_dly = 8000
acq_trig_sour = rp.RP_TRIG_SRC_CHA_PE
N = 16384

In [ ]:
def walsh(freq,duty,dec=0,Amp=1,ncyc=1,nor=1,period=1,trig_lvl=0.1,trig_dly=8000):
    
    rp.rp_GenReset()
    rp.rp_AcqReset()

    wave_form=generate_single_pulse_wave(dcyc=duty,Ampl=Amp) 
    wave_array=np.array(wave_form,dtype=np.float32) 

    rp.rp_GenWaveform(channel, rp.RP_WAVEFORM_ARBITRARY)
    rp.rp_GenArbWaveformNP(channel, wave_array)
    rp.rp_GenFreqDirect(channel,freq)

    rp.rp_GenMode(channel, rp.RP_GEN_MODE_BURST)
    rp.rp_GenBurstCount(channel, ncyc)                 
    rp.rp_GenBurstRepetitions(channel, nor)             
    rp.rp_GenBurstPeriod(channel, period)

    rp.rp_GenTriggerSource(channel, gen_trig_sour)
    rp.rp_GenOutEnableSync(True)


    ##### Acquisition #####

    rp.rp_AcqSetDecimation(dec)
    rp.rp_AcqSetTriggerLevel(rp.RP_CH_1, trig_lvl)
    rp.rp_AcqSetTriggerDelay(trig_dly)

    rp.rp_AcqStart()

    rp.rp_AcqSetTriggerSrc(acq_trig_sour)   
    rp.rp_GenTriggerOnly(channel)      
    while 1:
        trig_state = rp.rp_AcqGetTriggerState()[1]
        if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
            break
    while 1:
        if rp.rp_AcqGetBufferFillState()[1]:
            break

    fbuff = rp.fBuffer(N)
    res = rp.rp_AcqGetOldestDataV(rp.RP_CH_1, N, fbuff)[1]
    data_V = np.zeros(N, dtype = float)

    for i in range(0, N, 1):
        data_V[i] = fbuff[i]
    return data_V
    

In [ ]:
b=np.linspace(1,80,10)
c=np.multiply(b,1e6)

In [ ]:
f_amps=[]
for f in c:
    data_V=walsh(f,duty,dec,Amp)
    maxx=max(data_V)
    minn=min(data_V)
    Vpp=(maxx-minn)
    f_amps.append(Vpp)

print('done')

In [ ]:

tt= ((16384*dec)/250e6)* 1e6   # Time Frame; 64 micro s for dec=0 
x_ax=np.linspace(0,tt,len(data_V))

x_axx=x_ax[190:6000]
data_1=data_V[190:6000]

plt.plot(x_axx,data_1, label=f'Freq = {freq/1e6} MHz')
plt.xlabel('Time (microseconds)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
t_frame=x_axx
signal = data_1

signal_norm = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
t10_index = np.where(signal_norm >= 0.1)[0][0]
t90_index = np.where(signal_norm >= 0.9)[0][0]
rise_time = t_frame[t90_index] - t_frame[t10_index]
print(f"rise_time: {rise_time:.2e} microsec")

pulse width

In [ ]:
f_arr=[]
sig=abs(signal)

rise_threshold = 0.05
fall_threshold = 0.05
rising_edges = np.where((signal[:-1] < rise_threshold) & (signal[1:] >= rise_threshold))[0]
falling_edges=np.where((sig[1:] <= fall_threshold) & (sig[:-1] > fall_threshold))[0]

rise_times = t_frame[rising_edges]
fall_times=t_frame[falling_edges]
periods = abs(rise_times-fall_times)   # Time between successive rising edges
freq_p=duty/(periods)
print(freq_p)
mean_freq=np.mean(freq_p)
f_arr.append(mean_freq)